In [1]:
import torch.nn   as nn
import numpy      as np
import torch
import json

from libraries.model   import nGCNN, eGCNN, denoise, get_random_graph
from libraries.dataset import revert_standardize_dataset
from libraries.graph   import POSCAR_graph_encoding

import sys
sys.path.append('../')
import MP.MP_library as MPL

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# From random noise, we generate completely new materials
# A target property can be seeked with this approach

In [3]:
# Define folder in which all data will be stored
target_folder    = 'models/GM_BiSI'
edge_model_name = f'{target_folder}/edge_model.pt'
node_model_name = f'{target_folder}/node_model.pt'

# Number of graphs to predict
N_predictions = 10

# Amount of noise for the generative process
sigma = 0.1

# Define target to be generated
target_tensor = None

# Load model data

In [4]:
# Read the file in JSON format to a dictionary
with open(f'{target_folder}/model_parameters.json', 'r') as json_file:
    model_parameters = json.load(json_file)

# Number of diffusing and denoising steps
n_t_steps = model_parameters['n_t_steps']

# Decay of parameter alpha
noise_contribution = model_parameters['noise_contribution']
alpha_decay = 0.5 * (1 - noise_contribution**2)

# Dropouts for node and edge models (independent of each other)
dropout_node = model_parameters['dropout_node']
dropout_edge = model_parameters['dropout_edge']

# Generation of graph database for training

Load the datasets, already standarized if possible.

In [5]:
labels_name                 = f'{target_folder}/labels.pt'
dataset_name                = f'{target_folder}/dataset.pt'
dataset_name_std            = f'{target_folder}/standardized_dataset.pt'
dataset_parameters_name_std = f'{target_folder}/standardized_dataset_parameters.json'  # Parameters for rescaling the predictions

# Load the standardized dataset, with corresponding labels and parameters
dataset = torch.load(dataset_name_std)
labels  = torch.load(labels_name)

# Read the file in JSON format to a dictionary
with open(dataset_parameters_name_std, 'r') as json_file:
    dataset_parameters = json.load(json_file)

In [6]:
# Calculate the mean and standard deviation of the number of nodes
total_nodes = torch.tensor([data.num_nodes for data in dataset])
mean_nodes  = torch.mean(total_nodes.float()).item()
std_nodes   = torch.std(total_nodes.float()).item()

mean_nodes, std_nodes

(72.0, 0.0)

# Loading the model

In [7]:
# Determine number of features in dataset, considering the t_step information
n_features = dataset[0].num_node_features + 1

# Instantiate the models for nodes and edges
node_model = nGCNN(n_features, dropout_node).to(device)
node_model.load_state_dict(torch.load(node_model_name))
node_model.eval()

edge_model = eGCNN(n_features, dropout_edge).to(device)
edge_model.load_state_dict(torch.load(edge_model_name))
edge_model.eval()

print('\nNode GCNN:')
print(node_model)
print('\nEdge GCNN:')
print(edge_model)


Node GCNN:
nGCNN(
  (conv1): GraphConv(5, 256)
  (conv2): GraphConv(256, 5)
)

Edge GCNN:
eGCNN(
  (linear1): Linear(in_features=6, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=1, bias=True)
)


# Generating new cystals

In [8]:
# Predicting loop
predicted_dataset = []
with torch.no_grad():
    for i in range(N_predictions):
        # Get random number of nodes
        n_nodes = int(np.random.normal(mean_nodes, std_nodes))
        
        # Get random graph, acting as diffused
        diffused_graph = get_random_graph(n_nodes, n_features-1)
        
        # Denoise the diffused graph
        #print(f'Denoising...')
        denoised_graph, _ = denoise(diffused_graph, n_t_steps, node_model, edge_model,
                                    s=alpha_decay, sigma=sigma, target=target_tensor)
        
        # Append generated graph
        predicted_dataset.append(denoised_graph)

# Revert stardadization
denoised_graphs = revert_standardize_dataset(predicted_dataset, dataset_parameters)
denoised_graphs

[Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556])]

In [9]:
lattice_vectors = np.array([[12, 0, 0],
                            [0, 17, 0],
                            [0, 0, 10]])
for i in range(N_predictions):
    print()
    print(i+1)
    graph = denoised_graphs[i].clone()
    try:
        POSCAR_graph_encoding(graph, lattice_vectors, file_name=f'POSCAR-{i}', POSCAR_directory='./')
    except SystemExit:
        continue


1
6 34 51 3.9683318 4.2198825 4.2909393 1.9079686335276842 3.763916033617075

2
53 58 66 3.9628825 4.19982 4.3164225 1.8561518409550293 3.7673848656088764

3
3 16 67 4.135252 3.9470856 4.398083 1.6125568719214631 3.602658076798378

4
10 30 70 4.0659122 4.17059 4.220621 1.981329058213144 3.669898564820485

5
6 52 67 4.2347274 4.0536833 4.204234 1.9705730825872902 3.5424835734825955

6
31 39 68 4.0385537 4.448844 4.052917 2.436019649995078 3.7226362714639616

7
33 38 39 4.030197 4.175176 4.264099 1.9219957461202968 3.706484612455335

8
32 50 51 4.3014874 3.9877 4.2197437 1.9293684445996504 3.489883748690338

9
18 60 68 3.9393504 4.150427 4.358319 1.7451585131772733 3.7656958085429166

10
9 39 62 4.007904 4.084174 4.552939 1.4988533035577924 3.799199562078272
